# RNN, LSTM

In [ ]:
import numpy as np
from rnn_utils import *
from public_tests import *

## 1. Forward Propagation for the Basic RNN
* Xét VD RNN có $T_x=T_y$

  ![](images/RNN.png)

* Input $x$:
    * Với một single time step, $x^{(i)(t)}$ là $i^{th}$ sample tại time step $t$, là một 1D vector. VD với một 5000-word vocabulary, một word có thể đc biểu diễn bằng một 5000D vector.
    * $n_x$ là số lượng units (nghĩa là số features) trong một single time step của một training example.
* Time steps
    * Một RNN có nhiều time step, đc đánh chỉ số bằng $t$.
    * $T_x$ là số lượng time steps trong longest sequence.
* Một mini-batch có shape là $\left(n_x, m, T_x\right)$, với $m$ là batch size. 
* Khác với data ảnh khi ta lặp theo sample, mỗi sample có size (h, w), đối với NLP ta lặp theo time step. Tại một time step, ta dùng một 2D slice có shape $\left(n_x, m\right)$, chính là $x^{<t>}$
* Hidden state $a$:
    * Hidden state của một single training example là một 1D vector dài $n_a$.
    * Shape của hidden state cho một mini-batch là $\left(n_a, m\right)$, chính là $a^{<t>}$.
    * Nếu tính cả time step thì shape là $\left(n_a, m, T_x\right)$
* Prediction $\hat{y}$:
    * Là một 3D tensor có shape $\left(n_y, m, T_y\right)$, với $n_y$ là số lượng units trong vector biểu diễn prediction (VD là các XS để chọn kí tự), $T_y$ là số lượng time steps trong prediction.
    * Với một single time step, $\hat{y}^{<t>}$ có shape $\left(n_y, m\right)$.

### 1.1. RNN Cell
* RNN cell trong một single time step là: 

  ![](images/rnn_step_forward_figure2_v3a.png)
    
* Một RNN cell có input là current input $x^{<t>}$ và previous hidden state $a^{<t-1>}$ (chứa thông tin từ QK), và output ra $a^{<t>}$ để đưa vào RNN cell tiếp theo và cũng để dự đoán $\hat{y}^{<t>}$.
* Tính hidden state:   
    $$
    a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)
    $$
    $W_{aa}$ có shape $\left(n_a, n_a\right)$, $W_{ax}$ có shape $\left(n_a, n_x\right)$, $b_a$ có shape $\left(n_a, 1\right)$.
    
* Tính prediction:  
    $$
    \hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)
    $$
    $W_{ya}$ có shape $\left(n_y, n_a\right)$, $b_y$ có shape $\left(n_y, 1\right)$

In [ ]:
def rnn_cell_forward(xt, a_prev, parameters):
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
   
    # compute next activation state using the formula given above
    q=np.dot(Waa,a_prev) + np.dot(Wax,xt) + ba
    a_next = np.tanh(np.dot(Waa,a_prev) + np.dot(Wax,xt) + ba)

    # compute output of the current cell using the formula given above
    yt_pred = softmax( np.dot(Wya,a_next) + by)
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

### 1.2. RNN Forward Pass
Dùng lại RNN cell theo time step, mỗi lần dùng cùng các parameters $W_{ax}, W_{aa}, b_a, W_{ya}, b_y$ .   
  
  ![](images//rnn_forward_sequence_figure3_v3a.png)
    
Các bước thực hiện:
  1. Khởi tạo một ma trận 3D $a$ bằng 0, có shape $\left(n_a, m, T_x\right)$, để lưu tất cả hidden states.
  2. Khởi tạo một ma trận 3D $\hat{y}$ bằng 0, có shape $\left(n_y, m, T_y\right)$, để lưu các predictions.
  3. Khởi tạo $a^{<0>}$ random, có shape $\left(n_a, m\right)$, và đặt $a^{<t>}=a^{<0>}$ (là `a_next` trong code).
  4. Tại mỗi time step $t$:
      - Lấy $x^{<t>}$ là một 2D slice of $x$ tại time step $t$, có shape $\left(n_x, m\right)$
      - Chạy `rnn_cell_forward` để cập nhật $a^{<t>}$ và tính $\hat{y}^{<t>}$.
      - Lưu $a^{<t>}$ vào $a$ tại vị trí $t$.
      - Lưu $\hat{y}^{<t>}$ vào $\hat{y}$ tại vị trí $t$.

In [ ]:
def rnn_forward(x, a0, parameters):
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    # initialize "a" and "y_pred" with zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

### Notes:
* RNN có vấn đề vanishing gradients.
* RNN hoạt động tốt nhất khi mỗi output $\hat{y}^{<t>}$ có thể đc ước lượng bằng local context. Local context chỉ các information gần với time step $t$ tại prediction, cụ thể là các inputs $x^{<t'>}$ với $t'$ gần $t$.

## 2. LSTM
### 2.1. LSTM Cell
Một LSTM cell có cấu trúc:
   
![](images/LSTM_figure4_v3a.png)
    
Một LSTM cell theo dõi và update một cell state, hay memory variable $c^{<t>}$ tại mọi time step, khác với $a^{<t>}$.

### Forget gate $\Gamma_f$
Là một tensor chứa các values từ $0$ đến $1$. Nếu một unit trong forget gate có giá trị gần $0$, LSTM sẽ *quên* stored state trong unit tương ứng của previous cell state. Nếu gần $1$ thì LSTM sẽ gần như nhớ giá trị tương ứng trong stored state.
$$
\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)
$$
$\mathbf{W}_f$ có shape $\left(n_a, n_a+n_x\right)$, $\mathbf{b}_f$ có shape $\left(n_a, 1\right)$, $\Gamma_f$ có shape $\left(n_a, m\right)$, giống với shape của previous cell state $c^{<t-1>}$, để hai cái có thể nhân element-wise. \
$\mathbf{W}_f$ kiểm soát forget gate’s behavior. Nhân $\Gamma_f^{<t>}
*c^{<t>}$ giống như áp dụng một mask vào previous cell state. Nếu một giá trị trong $\Gamma_f^{<t>}$ gần $0$, thì tích sẽ gần $0$, khiến cho information đc lưu trong unit tương ứng ở $c^{<t>}$ ko đc nhớ cho next time step. Tương tự khi gần $1$, information sẽ đc dùng ở next time step
    
### Candidate value $\tilde{c}^{<t>}$
Là một tensor chứa thông tin từ current time step mà *có thể* sẽ đc lưu trong current cell state $c^{<t>}$. Phần candidate value mà đc đi qua sẽ phụ thuộc vào update gate.
$$
\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right)
$$
$\mathbf{W}_c$ có shape $\left(n_a, n_a+n_x\right)$, $\mathbf{b}_c$ có shape $\left(n_a, 1\right)$, $\tilde{c}^{<t>}$ có shape $\left(n_a, m\right)$.
    
### Update gate $\Gamma_i$
Đc dùng để xđ phần nào của candidate $\tilde{c}^{<t>}$ để cộng vào cell state $c^{<t>}$. Khi một unit trong update gate có giá trị gần $1$, nó cho phép giá trị trong $\tilde{c}^{<t>}$ đc truyền vào $c^{<t>}$. Ngược lại khi gần $0$.
$$
\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)
$$
$\mathbf{W}_i$ có shape $\left(n_a, n_a+n_x\right)$, $\mathbf{b}_i$ có shape $\left(n_a, 1\right)$, $\Gamma_i^{<t>}$ có shape $\left(n_a, m\right)$.
    
### Cell state $c^{<t>}$
Là memory đc truyền tới các future time steps. New cell state là kết hợp của previous cell state và candidate value.
$$
\mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle}
$$
Previous cell state $c^{<t-1>}$ đc điều chỉnh bằng forget gate $\Gamma_f^{<t>}$, còn candidate value $\tilde{c}^{<t>}$ đc điều chỉnh bằng update gate $\Gamma_i^{<t>}$. 

### Output gate $\Gamma_o$
Quyết định cái nào sẽ đc gửi đi để tính prediction.
$$
\mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})
$$

$\mathbf{W}_o$ có shape $\left(n_a, n_a+n_x\right)$, $\mathbf{b}_o$ có shape $\left(n_a, 1\right)$, $\Gamma_o^{<t>}$ có shape $\left(n_a, m\right)$.
    
### Hidden state $a^{<t>}$
Đc truyền tới các cell’s next time step và để tính ba gates trong next step, cũng như để tính prediction $\hat{y}^{<t>}$.
$$
\mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})
$$
Output gate giống như một mask để giữ hoặc ko giữ giá trị của $\tanh(\mathbf{c}^{\langle t \rangle}$ trong $a^{<t>}$. 
    
### Prediction $\hat{y}^{<t>}$
Trong case này prediction là classification, nên ta dùng softmax:
$$
\hat{\mathbf{y}}^{\langle t \rangle} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})
$$
$\mathbf{W}_y$ có shape $\left(n_y, n_a\right)$, $\mathbf{b}_y$ có shape $\left(n_y, 1\right)$, $\hat{y}^{<t>}$ có shape $\left(n_y, m\right)$.

In [ ]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"] # forget gate weight
    bf = parameters["bf"]
    Wi = parameters["Wi"] # update gate weight (notice the variable name)
    bi = parameters["bi"] # (notice the variable name)
    Wc = parameters["Wc"] # candidate value weight
    bc = parameters["bc"]
    Wo = parameters["Wo"] # output gate weight
    bo = parameters["bo"]
    Wy = parameters["Wy"] # prediction weight
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    # Concatenate a_prev and xt (≈1 line)
    concat = np.concatenate([a_prev,xt])

    # Compute values for ft, it, cct, c_next, ot, a_next using the formulas given figure (4) (≈6 lines)
    ft = sigmoid(np.dot(Wf,concat) + bf) # Forget Gate
    it = sigmoid(np.dot(Wi,concat) + bi) # Update Gate
    cct = np.tanh(np.dot(Wc,concat) + bc) # Candidate Value
    c_next = c_prev*ft + cct*it # C_t
    ot = sigmoid(np.dot(Wo,concat) + bo) # output gate
    a_next = ot*(np.tanh(c_next)) #a_t
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = softmax(np.dot(Wy,a_next) + by)

    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

### 2.2. Forward Pass for LSTM
Lặp lại các LSTM cell để xử lí một sequence of $T_x$ inputs:
    
![](images/LSTM_rnn.png)
    
Các bước thực hiện:
1. Khởi tạo các 3D tensors: hidden state $a$ có shape $\left(n_a, m, T_x\right)$, cell state $c$ có shape $\left(n_a, m, T_x\right)$, prediction $y$ có shape $\left(n_y, m, T_x\right)$.
2. Khởi tạo 2D tensor $a^{<0>}$ có shape $\left(n_a, m\right)$ và gán $a^{<t>}=a^{<0>}$ (là `a_next` trong code).
3. Khởi tạo $c^{<t>}$ là vector không, có shape $\left(n_a, m\right)$ (là `c_next` trong code).
4. Tại mỗi time step $t$:
    - Lấy $x^{<t>}$ là một 2D slice of $x$ tại time step $t$, có shape $\left(n_x, m\right)$
    - Chạy lstm`_cell_forward` để cập nhật $a^{<t>}$, $c^{<t>}$ và tính $\hat{y}^{<t>}$.
    - Lưu $a^{<t>}$ vào $a$, $c^{<t>}$ vào $c$, và $\hat{y}^{<t>}$ vào $\hat{y}$ tại vị trí $t$.

In [ ]:
def lstm_forward(x, a0, parameters):
    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    #Wy = parameters['Wy'] # Save parameters in local variables in case you want to use Wy instead of parameters['Wy']
    # Retrieve dimensions from shapes of x and parameters['Wy'] (≈2 lines)
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wy'].shape
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    a = np.zeros((n_a,m,T_x))
    c = np.zeros((n_a,m,T_x))
    y = np.zeros((n_y,m,T_x))
    
    # Initialize a_next and c_next (≈2 lines)
    a_next = a0
    c_next = np.zeros((n_a,m))
    
    # loop over all time-steps
    for t in range(T_x):
        # Get the 2D slice 'xt' from the 3D input 'x' at time step 't'
        xt = x[:,:,t]
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = c_next
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = yt
        # Append the cache into caches (≈1 line)
        caches.append(cache)
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return a, y, c, caches

### Notes:
* Các gates đều có giá trị thuộc $[0,1 ]$, candidate value và hidden state có giá trị thuộc $[-1, 1]$.
* LSTM khác RNN ở chỗ là nó dùng thêm một cell state, giống như một long-term memory, để xử lí vấn đề vanishing gradients.
* Forget gate sẽ xđ input units nào sẽ đc nhớ để truyền tiếp hoặc quên đi. Update gate sẽ xđ information nào sẽ bị bỏ đi hay thêm vào. Output gate sẽ xđ cái gì sẽ đc gửi đi để tính output.